In [1]:
import train as train
import networks as net
import models as model

import numpy as np
import tensorflow as tf
import os
import multiprocessing
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import scipy.stats as st
# suppress verbose
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

C:\Users\zhaoyu\.conda\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
_MULTIPROCESSING_CORE_COUNT = multiprocessing.cpu_count()
print("USE " + str(_MULTIPROCESSING_CORE_COUNT) + " THREADS ")

USE 16 THREADS 


In [3]:
dim = 1
nbSeg = 10
nbStep =1000
batchSize= 100

x0 = 100.0
xInit= x0*np.ones(dim,dtype=np.float32)   
Strike = 100.0
r = 0.05
sigma = 0.15
T=1
layerSize= [16,16,16,16,16]

num_epoch=100
num_epochExt=1 # modify training iterations
num_epochExt1 =1 # modify training iterations
nTest =2 # run times
graphLoc  = "./totF"

signature_test = np.load("sig_{}_from_{}_dim_{}_r_{}_sigma_{}_x_{}_K_{}.npy".format(nbSeg, nbStep,dim,  r, sigma, x0, Strike))[:batchSize]
sig_dim = signature_test.shape[-1]

In [4]:
# create the model
model = model.Model(dim,Strike, r, sigma, T, nbStep, nbSeg, delay=0)

# create network
theNetwork = net.FeedForwardUDU(dim, sig_dim,layerSize,tf.nn.elu)
    
# create graph
resol =  train.Base(xInit, model, Strike, r, sigma, T, nbStep, nbSeg, batchSize, theNetwork , initialLearningRate= 0.01)

print("Dim =", dim, " T =", T , " nbStep =" ,  nbStep , " batchsize =",batchSize,  " num_epoch =" , num_epoch, "  num_epochExt =",   num_epochExt,  " graph ", graphLoc)

Dim = 1  T = 1  nbStep = 1000  batchsize = 100  num_epoch = 100   num_epochExt = 1  graph  ./totF


In [5]:
print(os.getcwd())
directory = os.getcwd()

C:\Users\zhaoyu\Desktop\sig_american-main


In [6]:
Y0List = []
for i in tqdm(range(nTest)):
    Y0, Z0    = resol.BuildAndtrainML(num_epoch=num_epoch, num_epochExt=num_epochExt,num_epochExt1 =num_epochExt1, nbOuterLearning=4 )
    print(" EstimMC Val is " , Y0,    " Z0 ", Z0)
    Y0List.append(Y0)
    yList = np.array(Y0List)
    Y0_mean = np.mean(yList)
    with open(r"{}\y_d{}_seg{}.csv".format(directory, dim, nbSeg ), 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([Y0, Z0, Y0_mean])

 50%|██████████████████████████████████████████                                          | 1/2 [00:06<00:06,  6.16s/it]

 EstimMC Val is  4.24304  Z0  [[0.9782917]]


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.17s/it]

 EstimMC Val is  4.4616504  Z0  [[0.98671496]]


In [7]:
yList = np.array(Y0List)
yMean = yList.mean()
print("EstimMC Val ", yMean, " 95% CI ", st.t.interval(alpha=0.95, df=len(yList)-1, loc=np.mean(yList), scale=st.sem(yList)) )

EstimMC Val  4.3523455  95% CI  (2.9634919322961424, 5.741199000931397)
